In [1]:
import argparse
import os

import numpy as np
import tensorflow as tf

from dataset import KinQueryDataset, preprocess

In [2]:
def _batch_loader(iterable, n=1):
    """
    데이터를 배치 사이즈만큼 잘라서 보내주는 함수입니다. PyTorch의 DataLoader와 같은 역할을 합니다

    :param iterable: 데이터 list, 혹은 다른 포맷
    :param n: 배치 사이즈
    :return:
    """
    length = len(iterable)
    for n_idx in range(0, length, n):
        yield iterable[n_idx:min(n_idx + n, length)]


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [3]:
# User options
batch = 10
epochs = 100

embedding = 30
strmaxlen = 200
DATASET_PATH = '../sample_data/movie_review/'

# 모델의 specification
input_size = embedding*strmaxlen
output_size = 1
hidden_layer_size = 200
learning_rate = 0.001
character_size = 251

x = tf.placeholder(tf.int32, [None, strmaxlen])
y_ = tf.placeholder(tf.float32, [None, output_size])
# 임베딩
char_embedding = tf.get_variable('char_embedding', [character_size, embedding])
embedded = tf.nn.embedding_lookup(char_embedding, x)

# 첫 번째 레이어
cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer_size)
cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=0.9)
#tf.cast(cell, tf.int32)
#tf.cast(x, tf.int32)

output, state = tf.nn.dynamic_rnn(cell, embedded, dtype=tf.float32)
output = tf.transpose(output, [1, 0, 2])
output = output[-1]
print("output shaep",output.shape)

# 두 번째 (아웃풋) 레이어
second_layer_weight = weight_variable([hidden_layer_size, output_size])
second_layer_bias = bias_variable([output_size])
foutput = tf.matmul(output, second_layer_weight) + second_layer_bias
#output_sigmoid = tf.sigmoid(output)
print("output shaep",foutput.shape)
print("lables shaep",y_.shape)
# loss와 optimizer
#global_step = tf.Variable(0)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=foutput, labels=y_))
ls = tf.reduce_mean(tf.losses.mean_squared_error(labels=y_, predictions=foutput))
#ls = tf.reduce_mean(tf.square(output-y_))

#print(loss)

#learning_rate= tf.train.exponential_decay(learning_rate, global_step, 10000, 0.75)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(ls)
#train_step = tf.train.AdadeltaOptimizer(1.0, 0.95, 1e-6).minimize(cost)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

output shaep (?, 200)
output shaep (?, 1)
lables shaep (?, 1)


In [4]:
dataset = KinQueryDataset(DATASET_PATH, strmaxlen)
dataset_len = len(dataset)

In [5]:
print(dataset_len)

107


In [6]:
#np.eye(2)[[0]]

array([[ 1.,  0.]])

In [7]:
#def label_one_hot(labels):
#    one_hot = (np.arange(output_size) == labels[:]).astype(np.int32)
#    return one_hot

In [8]:
#label_one_hot([1])

array([0], dtype=int32)

In [9]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()
#batch = 1
dataset = KinQueryDataset(DATASET_PATH, strmaxlen)
dataset_len = len(dataset)
one_batch_size = dataset_len//batch
if dataset_len % batch != 0:
    one_batch_size += 1
# epoch마다 학습을 수행합니다.
for epoch in range(epochs):
    avg_loss = 0.0
    avg_l = 0.0
    for i, (data, labels) in enumerate(_batch_loader(dataset, batch)):
        #zero = np.zeros([batch,1])
        #zero[0] = 
        _, loss, l = sess.run([train_step, cost, ls],
                           feed_dict={x: data, y_: labels})
        if math.isnan(loss):
            print('Detected NaN')
            import pdb; pdb.set_trace()
        #print('Batch : ', i + 1, '/', one_batch_size,', BCE in this minibatch: ', (loss))
        avg_loss += float(loss)
        avg_l += float(l)
    print('epoch:', epoch, ' train_loss:', float(avg_loss/one_batch_size),float(avg_l/one_batch_size),timeSince(start))

epoch: 0  train_loss: 0.0 49.351018992337316 0m 1s
epoch: 1  train_loss: 0.0 8.865284220738845 0m 2s
epoch: 2  train_loss: 0.0 6.656276709654114 0m 3s
epoch: 3  train_loss: 0.0 6.765658248554576 0m 4s
epoch: 4  train_loss: 0.0 7.06935042142868 0m 5s
epoch: 5  train_loss: 0.0 7.043845360929316 0m 6s
epoch: 6  train_loss: 0.0 6.552712960676714 0m 7s
epoch: 7  train_loss: 0.0 6.6933041485873135 0m 8s
epoch: 8  train_loss: 0.0 6.661902151324532 0m 9s
epoch: 9  train_loss: 0.0 6.757591068744659 0m 10s
epoch: 10  train_loss: 0.0 6.653268927877599 0m 11s
epoch: 11  train_loss: 0.0 6.702085760506717 0m 12s
epoch: 12  train_loss: 0.0 6.543912156061693 0m 13s
epoch: 13  train_loss: 0.0 6.695739312605425 0m 14s
epoch: 14  train_loss: 0.0 6.382793155583468 0m 15s
epoch: 15  train_loss: 0.0 6.655661680481651 0m 16s
epoch: 16  train_loss: 0.0 6.748770670457319 0m 17s
epoch: 17  train_loss: 0.0 6.814165873961016 0m 18s
epoch: 18  train_loss: 0.0 6.722245622764934 0m 19s
epoch: 19  train_loss: 0.0 6.5